In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#!/home/ektov1-av_ca-sbrf-ru/bin/python35
import os
import sys
curruser = os.environ.get('USER')

sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

sys.path.insert(0, './src')

# import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

# import logging
# logging.basicConfig(filename='./__upd_runGASiteProd2SAS__.log',level=logging.INFO,
#                     format='%(asctime)s %(levelname)s %(name)s %(message)s')
# logger = logging.getLogger(__name__)


import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime

from spark_connector import SparkConnector
from sparkdb_loader import spark
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
from etl import ETLORA


# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))


##-----------------------------------------
conn_schema = 'sbx_team_digitcamp'
ga_schema = 'sklod_external_google_analytics'
tbl = 'GA_SITE_ALL_PRODS'
out_table_name = 'GA_ALL_SCENARIOS_HIST'
##-----------------------------------------http://pklis-chd000224.labiac.df.sbrf.ru:8100/notebooks/ISKRA_GET_LOAD/Hive2Iskra.ipynb#

def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

# @typed_udf(StringType())
def match_product(product_dict_nm):
    def get_product(product_norm, product_dict_nm):
        if product_norm is None:
            return None

        for product_nm in product_dict_nm:
            if product_nm in product_norm:
                return product_dict_nm[product_nm]
            else:
                return None

    return f.udf(lambda x: get_product(x,product_dict_nm))

In [3]:
print("### Starting spark context. Run!")

sp = spark(schema=conn_schema,
           dynamic_alloc=False,
           numofinstances=10, 
           numofcores=8,
           kerberos_auth=True,
           process_label="shaymanov-an"
           )
hive = sp.sql

print(sp.sc.version)

### Starting spark context. Run!
2.4.0.cloudera2


In [10]:
table_name = 'GA_ALL_SCENARIOS_HIST'
sdf = hive.sql('''select * from {}.{} 
                  where 1=1 and
                        inn = '260800070346' and
                        channel = 'SBBOL' and
                        id_product = 'DEPOSIT' and
                        (load_dt >= timestamp('2021-03-01') ) and 
                        (load_dt <= timestamp('2021-03-16') )'''.format(conn_schema,table_name))

In [ ]:
sdf.show(40)

## Upload to ISKRA from Pandas df

In [6]:
df= sdf.toPandas()

In [ ]:
df.dtypes

In [122]:
cols = [col for col in df.columns if col != "commonSegmentoUID"]+["commonSegmentoUID"]
df = df.loc[:,cols]

## Create empty table with predefined schema

In [135]:
str_ = 'CREATE TABLE {} ( '
for column_name, column in df.iteritems():
    if column.dtype.kind == 'O':
        if (df[column_name].str.len().max() >= 4000) or ('COMMONSEGMENTOUID' in column_name.upper()):
#             df.drop(columns=[column_name], inplace=True)
            str_+=column_name.upper() + ' ' +'CLOB, '
        else:
            if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
                str_+=column_name.upper() + ' ' +'VARCHAR2(20), '
            else:
                str_+=column_name.upper() + ' ' +'VARCHAR2(800), '
    elif column.dtype.kind == 'i':
        str_+=column_name.upper() + ' ' +'NUMBER(20), '
    elif column.dtype.kind == 'f':
        df[column_name] = df[column_name].fillna(0.0)
        str_+=column_name.upper() + ' ' +'FLOAT(126), '        
    elif column.dtype.kind == 'M':
        str_+=column_name.upper() + ' ' +'TIMESTAMP, '
    else:
        None  
res=str_.strip()[:-1] + ' )'
res

'CREATE TABLE {} ( INN VARCHAR2(20), SBBOLUSERID VARCHAR2(800), ID_CRM VARCHAR2(800), SCENARIO_ID VARCHAR2(800), CHANNEL VARCHAR2(800), ID_PRODUCT VARCHAR2(800), MADEAL_PRODUCT_NORM VARCHAR2(800), MADEAL_PROD_TYPE_NAME VARCHAR2(800), MADEAL_APPL_CORE_TXT VARCHAR2(800), MADEAL_COMPLETE_DT TIMESTAMP, HASPRODUCT NUMBER(20), ESSENSE VARCHAR2(800), RETURNCNT NUMBER(20), NUMBEROFSTEPS NUMBER(20), FUNNEL_RATE VARCHAR2(800), SUM_OFFER_PRIORITY NUMBER(20), SUM_OFFER_NONTOP NUMBER(20), MINSESSIONSTARTTIME TIMESTAMP, MAXSESSIONSTARTTIME TIMESTAMP, LOAD_DT TIMESTAMP, CTL_LOADING NUMBER(20), COMMONSEGMENTOUID CLOB )'

In [3]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('tech_iskra[iskra]', 'Uthvfy123', ld._get_dsn('iskra4')) 
db.connect() 

In [4]:
sql = ''' select /*+ parallel (12) */ 
                 min(interaction_start_date),
                 max(interaction_start_date) 
          from AIST_INTERACTION_HISTORY '''
df2 = pd.read_sql(sql, con=db.connection)

In [5]:
df2.head()

,MIN(INTERACTION_START_DATE),MAX(INTERACTION_START_DATE)
0,2019-01-04 00:07:43,2021-06-24 20:57:10


In [137]:
table_name = 'tmp_GA_ALL_SCENARIOS_HIST'

In [138]:
sql = "DROP TABLE {}".format(table_name)
db.cursor.execute(sql)
db.connection.commit()
print('dropping done')

dropping done


In [139]:
sql = res.format(table_name)

db.cursor.execute(sql)
db.connection.commit()
print('creating done')

creating done


In [140]:
# table_name = 'tmp_GA_ALL_SCENARIOS_HIST'
ld.upload_df_or_csv(df, table_name, parallel=1, 
                    password='Uthvfy123', path= None,
                    isclobe=1, isuseclobdct=1, verbose=1, njobs=5)

OrderedDict([('INN', <cx_Oracle.DbType DB_TYPE_LONG>), ('SBBOLUSERID', <cx_Oracle.DbType DB_TYPE_LONG>), ('ID_CRM', <cx_Oracle.DbType DB_TYPE_LONG>), ('SCENARIO_ID', <cx_Oracle.DbType DB_TYPE_LONG>), ('CHANNEL', <cx_Oracle.DbType DB_TYPE_LONG>), ('ID_PRODUCT', <cx_Oracle.DbType DB_TYPE_LONG>), ('MADEAL_PRODUCT_NORM', <cx_Oracle.DbType DB_TYPE_LONG>), ('MADEAL_PROD_TYPE_NAME', <cx_Oracle.DbType DB_TYPE_LONG>), ('MADEAL_APPL_CORE_TXT', <cx_Oracle.DbType DB_TYPE_LONG>), ('MADEAL_COMPLETE_DT', <cx_Oracle.DbType DB_TYPE_TIMESTAMP>), ('HASPRODUCT', <cx_Oracle.ApiType NUMBER>), ('ESSENSE', <cx_Oracle.DbType DB_TYPE_LONG>), ('RETURNCNT', <cx_Oracle.ApiType NUMBER>), ('NUMBEROFSTEPS', <cx_Oracle.ApiType NUMBER>), ('FUNNEL_RATE', <cx_Oracle.DbType DB_TYPE_LONG>), ('SUM_OFFER_PRIORITY', <cx_Oracle.ApiType NUMBER>), ('SUM_OFFER_NONTOP', <cx_Oracle.ApiType NUMBER>), ('MINSESSIONSTARTTIME', <cx_Oracle.DbType DB_TYPE_TIMESTAMP>), ('MAXSESSIONSTARTTIME', <cx_Oracle.DbType DB_TYPE_TIMESTAMP>), ('LOAD_D

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 out of   6 | elapsed:  1.3min finished


## Check saved DF via cx_Oracle

In [141]:
query = '''
select /*+ parallel (12) */
          *
FROM {}
where rownum < 1000
'''.format(table_name)

In [142]:
df2 = pd.read_sql(query, con=db.connection)

In [ ]:
df2.head()

## Check saved DF via JDBC driver

In [26]:
print("\n,".join([col for col in df2.columns]))

INN
,SBBOLUSERID
,COMMONSEGMENTOUID
,ID_CRM
,SCENARIO_ID
,CHANNEL
,ID_PRODUCT
,MADEAL_PRODUCT_NORM
,MADEAL_PROD_TYPE_NAME
,MADEAL_APPL_CORE_TXT
,MADEAL_COMPLETE_DT
,HASPRODUCT
,ESSENSE
,RETURNCNT
,NUMBEROFSTEPS
,FUNNEL_RATE
,SUM_OFFER_PRIORITY
,SUM_OFFER_NONTOP
,MINSESSIONSTARTTIME
,MAXSESSIONSTARTTIME
,LOAD_DT
,CTL_LOADING


In [71]:
query = '''
(select /*+ parallel (12) */
    INN
    ,SBBOLUSERID
    ,COMMONSEGMENTOUID
    ,ID_CRM
    ,SCENARIO_ID
    ,CHANNEL
    ,ID_PRODUCT
    ,MADEAL_PRODUCT_NORM
    ,MADEAL_PROD_TYPE_NAME
    ,MADEAL_APPL_CORE_TXT
    ,CAST(MADEAL_COMPLETE_DT as VARCHAR2(40))
    ,HASPRODUCT
    ,ESSENSE
    ,RETURNCNT
    ,NUMBEROFSTEPS
    ,FUNNEL_RATE
    ,SUM_OFFER_PRIORITY
    ,SUM_OFFER_NONTOP
    ,MINSESSIONSTARTTIME
    ,MAXSESSIONSTARTTIME
    ,LOAD_DT
    ,CTL_LOADING       
FROM {}
where rownum < 1000)
'''.format(table_name)

sdf = sp.get_oracle(OracleDB('iskra4'), query)

In [ ]:
sdf.show()

## Extract via JDBS driver

In [ ]:
## !!! Don't forget to created an empty table before with CLOB types followed by all non-clob columns

In [129]:
table_name = 'GA_ALL_SCENARIOS_HIST'
sdf = hive.sql('''select * from {}.{} where (load_dt >= timestamp('2021-03-01') ) and 
                                            (load_dt <= timestamp('2021-03-06') )'''.format(conn_schema,table_name))

In [130]:
sdf = sdf.select([col for col in sdf.columns if col not in ("commonSegmentoUID")]+["commonSegmentoUID"])

In [131]:
sdf = sdf.select([col.upper() for col in sdf.columns])

typesmap={}
for column_name, column in sdf.dtypes:
    if column == 'string':
        if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
            typesmap[column_name] = 'VARCHAR(20)'
        elif 'commonSegmentoUID'.upper() in column_name.upper():
            typesmap[column_name] = 'CLOB'
        else:
            typesmap[column_name] = 'VARCHAR(900)'
    elif column == 'int':
        typesmap[column_name] = 'INTEGER'
    elif column == 'bigint':
        typesmap[column_name] = 'LONG'
    elif column == 'timestamp':
        typesmap[column_name] = 'TIMESTAMP'
    elif column == 'float' or column == 'double':
        typesmap[column_name] = 'FLOAT'
    else:
        None
        
cols = ', '.join([col + ' ' + typesmap[col] for col in sdf.columns])
cols

'INN VARCHAR(20), SBBOLUSERID VARCHAR(900), ID_CRM VARCHAR(900), SCENARIO_ID VARCHAR(900), CHANNEL VARCHAR(900), ID_PRODUCT VARCHAR(900), MADEAL_PRODUCT_NORM VARCHAR(900), MADEAL_PROD_TYPE_NAME VARCHAR(900), MADEAL_APPL_CORE_TXT VARCHAR(900), MADEAL_COMPLETE_DT TIMESTAMP, HASPRODUCT INTEGER, ESSENSE VARCHAR(900), RETURNCNT INTEGER, NUMBEROFSTEPS INTEGER, FUNNEL_RATE VARCHAR(900), SUM_OFFER_PRIORITY INTEGER, SUM_OFFER_NONTOP INTEGER, MINSESSIONSTARTTIME TIMESTAMP, MAXSESSIONSTARTTIME TIMESTAMP, LOAD_DT TIMESTAMP, CTL_LOADING LONG, COMMONSEGMENTOUID CLOB'

In [132]:
table_name = "tmp_GA_ALL_SCENARIOS_HIST"

db = OracleDB('iskra4')
mode = 'append'
sdf \
    .write \
    .format('jdbc') \
    .mode(mode) \
    .option('url', 'jdbc:oracle:thin:@//'+db.dsn) \
    .option('user', db.user) \
    .option('password', db.password) \
    .option('dbtable', table_name) \
    .option('createTableColumnTypes', cols)\
    .option('driver', 'oracle.jdbc.driver.OracleDriver') \
    .save()

## Check saved DF

In [133]:
query = '''
(select /*+ parallel (12) */
    *      
FROM {}
where rownum < 1000)
'''.format(table_name)

sdf = sp.get_oracle(OracleDB('iskra4'), query)

In [ ]:
sdf.show()